In [1]:
#Suzan Iloglu, May 21,2020
import gurobipy as gp
from gurobipy import GRB
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200

In [2]:
from IPython.display import Image

## Social Vulnerability Index for Mapping Community Health Worker Need
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a>

### I. Importing SVI data and computing state percentile ranking
The CDC uses both a USA-wide percentile ranking and a state by state percentile ranking. For our project given that funding is likely going to be managed at a state level using a state percentile ranking makes the most sense and will be most sensitive to regional socioeconomic differences. 

In [3]:
## import SVI data
svi_census = gpd.read_file("Data/SVI2018_US_tract.geojson")
#svi_tribal = gpd.read_file("Data/SVI2018_tribal_tract.geojson")
svi_puerto_rico = gpd.read_file("Data/SVI2018_PuertoRico_tract.geojson")


In [4]:
## import svi data by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")
#svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)
svi_counties = svi_counties_mainland
svi_counties.head(5)


,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,geometry
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,71,12398,439,-999,-999,-999,-999,-999,-999,3669,426,7083,25,9318,14,6280,495,1330,285.0,34397,145.0,755,209.5,67,37.1,7770,431,264,77.1,763,160,654,142,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,13.8,1.6,18.0,0.1,23.7,0.0,16.1,1.3,10.7,2.3,87.5,0.4,2.1,0.6,0.3,0.2,38.8,2.2,2.1,0.6,6.2,1.3,1.7,0.4,-999.0000,-999.0000,-999.0000,0.5922,-999.0000,-999.0000,0.4893,0.6826,0.5610,0.8383,2.5712,0.8758,0.9917,0.7740,1.7657,0.9268,0.0551,0.9869,0.5498,0.5788,0.4126,2.5832,0.5409,-999.0000,-999.0000,-999,-999,-999,0,-999,0,0,0,0,0,1,0,1,0,1,0,0,0,1,-999,4160,588,10.6,1.5,32290,1.536344,6.452810,"POLYGON ((-107.62554 36.56587, -107.62523 36.5..."
1,01,ALABAMA,AL,Autauga,01001,"Autauga County, Alabama",594.443459,55200,0,23315,71,21115,383,8422,1137,1065,257,29372,2306,4204,475,8050,75,13369,32,10465,729,1586,319.9,13788,59.0,426,205.9,886,308.7,4279,469,299,142.3,1191,272,546,161,15.4,2.1,4.2,1.0,29372.0,2306.0,11.3,1.3,14.6,0.1,24.2,0.1,19.3,1.3,7.5,1.5,25.0,0.1,0.8,0.4,3.8,1.3,18.4,2.0,1.4,0.7,5.6,1.3,1.0,0.3,0.5401,0.2745,0.2860,0.4397,1.5403,0.3631,0.1850,0.7529,0.7905,0.3792,2.1076,0.5810,0.6336,0.5113,1.1449,0.5947,0.6017,0.7408,0.2964,0.4846,0.1525,2.2760,0.3741,7.0688,0.4354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3875,508,7.1,0.9,37301,0.150256,2.052740,"POLYGON ((-86.92120 32.65754, -86.92035 32.658..."
2,01,ALABAMA,AL,Blount,01009,"Blount County, Alabama",644.830460,57645,0,24222,55,20600,396,8220,992,909,193,22656,905,7861,727,10233,91,13468,53,8114,592,1437,267.2,7413,229.0,934,239.3,211,104.2,6108,476,339,130.7,856,201,543,117,14.4,1.7,4.1,0.8,22656.0,905.0,19.8,1.8,17.8,0.2,23.4,0.1,14.2,1.0,7.0,1.3,12.9,0.4,1.7,0.4,0.9,0.4,25.2,2.0,1.6,0.6,4.2,1.0,0.9,0.2,0.4723,0.2611,0.7561,0.8405,2.3300,0.6143,0.4715,0.6406,0.3763,0.2961,1.7845,0.3187,0.4206,0.7170,1.1376,0.5915,0.1512,0.8816,0.3703,0.2420,0.1165,1.7616,0.1741,7.0137,0.4242,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6303,732,11.0,1.3,40036,0.164403,2.392326,"POLYGON ((-86.96336 33.85822, -86.95967 33.857..."
3,01,ALABAMA,AL,Butler,01013,"Butler County, Alabama",776.838201,20025,0,10026,51,6708,274,4640,521,567,147,20430,1258,2141,268,3806,21,4566,88,3492,370,704,143.9,9641,22.0,93,137.4,134,47.4,2625,212,119,57.7,520,102,322,88,23.5,2.6,6.7,1.7,20430.0,1258.0,15.4,1.9,19.0,0.1,22.8,0.4,17.7,1.9,10.5,2.1,48.1,0.1,0.5,0.7,1.3,0.5,26.2,2.1,1.8,0.9,7.8,1.5,1.6,0.4,0.8860,0.6968,0.8879,0.6753,3.1460,0.8455,0.5928,0.5578,0.6845,0.8185,2.6536,0.9077,0.8711,0.3582,1.2293,0.6447,0.2416,0.8940,0.4457,0.7685,0.3792,2.7290,0.6259,9.7579,0.8653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2005,340,10.2,1.7,17280,0.191747,1.818327,"POLYGON ((-86.90894 31.96167, -86.87498 31.961..."
4,01,ALABAMA,AL,Calhoun,01015,"Calhoun County, Alabama",605.867251,115098,0,53682,184,45033,683,20819,1317,4628,526,24706

In [5]:
## replacing -999 values with nan for calculations
#svi_census = pd.concat([svi, svi_puerto_rico], sort = False)
svi_county = svi_counties.fillna(-999)
svi_census = svi_census.fillna(-999)
#svi_county.head(5)


In [6]:

#svi_county[svi_county['STATE'] == "PUERTO RICO"]


In [7]:
## calculating state by state percentile rank for tract svi
svi_census['RPL_ThemesStates'] = svi_census.groupby('STATE')['SPL_THEMES'].rank(pct=True)
svi_census['RPL_Theme1States'] = svi_census.groupby('STATE')['SPL_THEME1'].rank(pct=True)
svi_census['RPL_Theme2States'] = svi_census.groupby('STATE')['SPL_THEME2'].rank(pct=True)
svi_census['RPL_Theme3States'] = svi_census.groupby('STATE')['SPL_THEME3'].rank(pct=True)
svi_census['RPL_Theme4States'] = svi_census.groupby('STATE')['SPL_THEME4'].rank(pct=True)

svi_census.head(5)


,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,geometry,RPL_ThemesStates,RPL_Theme1States,RPL_Theme2States,RPL_Theme3States,RPL_Theme4States
0,01,ALABAMA,AL,01015,Calhoun,01015981901,"Census Tract 9819.01, Calhoun County, Alabama",24.243865,0,12,0,12,0,12,0,12,0,12,-999,-999,0,12,0,12,0,12,0,12,0,17.0,0,17.0,0,48.0,0,17.0,0,12,0,17.0,0,12,0,12,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0000,-999.0000,-999.000,-999.0,-999.000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,12,-999.0,-999.0,656,0.006105,0.339400,"POLYGON ((-86.01323 33.68426, -86.01224 33.687...",0.002966,0.002966,0.002542,0.002542,0.002966
1,01,ALABAMA,AL,01015,Calhoun,01015981902,"Census Tract 9819.02, Calhoun County, Alabama",34.646714,0,12,0,12,0,12,0,12,0,12,-999,-999,0,12,0,12,0,12,0,12,0,17.0,0,17.0,0,48.0,0,17.0,0,12,0,17.0,0,12,0,12,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0000,-999.0000,-999.000,-999.0,-999.000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,12,-999.0,-999.0,146,0.008752,0.420459,"POLYGON ((-86.01730 33.69662, -86.01704 33.698...",0.002966,0.002966,0.002542,0.002542,0.002966
2,01,ALABAMA,AL,01015,Calhoun,01015981903,"Census Tract 9819.03, Calhoun County, Alabama",13.926276,0,12,0,12,0,12,0,12,0,12,-999,-999,0,12,0,12,0,12,0,12,0,17.0,0,17.0,0,48.0,0,17.0,0,12,0,17.0,0,12,0,12,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.0,-999.0000,-999.0000,-999.000,-999.0,-999.000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,12,-999.0,-999.0,0,0.003508,0.330164,"POLYGON ((-85.78501 33.66553, -85.78318 33.666...",0.002966,0.002966,0.002542,0.002542,0.002966
3,01,ALABAMA,AL,01097,Mobile,01097003605,"Census Tract 36.05, Mobile County, Alabama",0.732300,899,316,0,12,0,12,0,12,40,39,2651,404,0,12,0,12,22,30,42,40,0,17.0,330,392.6,0,48.0,0,17.0,0,12,0,17.0,0,12,899,316,-999.0,-999.0,9.9,10.1,2651.0,404.0,-999.0,-999.0,0.0,3.5,2.4,3.2,4.7,4.3

In [8]:
# Create dictionary for census tract SVI
svi_census['FIPS'] = svi_census['FIPS'].astype(int)
svi_census['STCNTY'] = svi_census['STCNTY'].astype(int)
SVI_census = dict(zip(svi_census.FIPS, svi_census.RPL_ThemesStates))

In [9]:
## exporting tracts svi to geojson
# svi_census.to_file("Data/svi_2018_tracts_state_ranked.geojson",driver='GeoJSON')

In [10]:
## calculating state by state percentile rank for county svi

svi_county['RPL_ThemesStatesCt'] = svi_county.groupby('STATE')['SPL_THEMES'].rank(pct=True)
svi_county['RPL_Theme1StatesCt'] = svi_county.groupby('STATE')['SPL_THEME1'].rank(pct=True)
svi_county['RPL_Theme2StatesCt'] = svi_county.groupby('STATE')['SPL_THEME2'].rank(pct=True)
svi_county['RPL_Theme3StatesCt'] = svi_county.groupby('STATE')['SPL_THEME3'].rank(pct=True)
svi_county['RPL_Theme4StatesCt'] = svi_county.groupby('STATE')['SPL_THEME4'].rank(pct=True)
svi_county.head(1)

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,geometry,RPL_ThemesStatesCt,RPL_Theme1StatesCt,RPL_Theme2StatesCt,RPL_Theme3StatesCt,RPL_Theme4StatesCt
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,71,12398,439,-999,-999,-999,-999,-999,-999,3669,426,7083,25,9318,14,6280,495,1330,285.0,34397,145.0,755,209.5,67,37.1,7770,431,264,77.1,763,160,654,142,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,13.8,1.6,18.0,0.1,23.7,0.0,16.1,1.3,10.7,2.3,87.5,0.4,2.1,0.6,0.3,0.2,38.8,2.2,2.1,0.6,6.2,1.3,1.7,0.4,-999.0,-999.0,-999.0,0.5922,-999.0,-999.0,0.4893,0.6826,0.561,0.8383,2.5712,0.8758,0.9917,0.774,1.7657,0.9268,0.0551,0.9869,0.5498,0.5788,0.4126,2.5832,0.5409,-999.0,-999.0,-999,-999,-999,0,-999,0,0,0,0,0,1,0,1,0,1,0,0,0,1,-999,4160,588,10.6,1.5,32290,1.536344,6.45281,"POLYGON ((-107.62554 36.56587, -107.62523 36.5...",0.030303,0.030303,0.666667,0.515152,0.30303


In [11]:
# Create a dictionary for county SVI
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
SVI_county = dict(zip(svi_county.FIPS, svi_county.RPL_ThemesStatesCt))
print (SVI_county)

{35039: 0.030303030303030304, 1001: 0.1044776119402985, 1009: 0.07462686567164178, 1013: 0.7611940298507462, 1015: 0.6865671641791045, 1017: 0.5223880597014925, 1031: 0.22388059701492538, 1033: 0.08955223880597014, 1039: 0.6268656716417911, 1043: 0.16417910447761194, 1045: 0.582089552238806, 1051: 0.208955223880597, 1055: 0.34328358208955223, 1067: 0.2537313432835821, 1069: 0.40298507462686567, 1071: 0.2835820895522388, 1077: 0.11940298507462686, 1079: 0.14925373134328357, 1083: 0.19402985074626866, 1089: 0.04477611940298507, 1095: 0.4925373134328358, 1097: 0.5671641791044776, 1103: 0.43283582089552236, 1111: 0.47761194029850745, 1113: 0.8656716417910447, 1115: 0.05970149253731343, 1117: 0.014925373134328358, 1121: 0.7761194029850746, 2261: 0.13793103448275862, 4021: 0.3333333333333333, 5009: 0.12, 5011: 0.9866666666666667, 5033: 0.25333333333333335, 5037: 0.41333333333333333, 5045: 0.13333333333333333, 5047: 0.22666666666666666, 5051: 0.68, 5053: 0.09333333333333334, 5063: 0.48, 5083:

## Community Health Workers Coverage Model

### Objective and Prerequisites

In this model, we solve a version of capacitated coverage model: how to allocate community health workers to maximize the coverage in the census tract level. We consider 2-level allocation. First, we allocate community health care workers within counties per state and then census tracts within per county in the state. We implement this model in the Gurobi Python interface and compute optimal solution.

### Motivation
While there has been tremendous emphasis across the country on scaling up contact tracing for COVID-19, exposure notification is only one part of what is needed in communities hardest hit by the pandemic. We need a new politics of care, which addresses the larger health, social and economic issues that many places around the country were struggling with long before COVID-19 and that created health disparities that have lingered for generations. A community health worker approach, which combines basic public health tasks, like contact tracing, health education, social work and legal advocacy, are required to rebuild communities in the wake of COVID-19 and ensure better health for all for the long term.


### Problem Description
![Contact Tracing Allocation](img1.png)

### Model Formulation 
 
#### Sets
$I $ set of contact tracing demand points (census tracts)<br>
$J$  set of contact tracer center locations (counties)<br>
$S$  set of states<br>
$N_i$ set of contact tracer center locations that can serve demand point $i \in I$<br>
$M_j$ set of contact tracer center locations that belongs to state $s \in S$<br>

#### Parameters
$w_{i}$  weight assigned to demand point $i \in I$ <br>
$d_{i}$  total number of contact tracer needed at demand point $i \in I$<br>
$z_{j}$  weight assigned to contact tracer location  $j \in J$<br>
$c_{s}$  total number of contact tracer capacity at each state $s \in S$<br>


The decision variables for the model are as follows: <br>
 $y_{j} = $ the number of contact tracer at center $j \in J$. <br>
 $x_{ij} = $ the number of community health care workers assigned to demand point $i \in I$ by contact tracing center $j \in J$, and 0 otherwise. <br>



The integer programming formulation of our model is as follows.
<br>
$$
\begin{align}
    \max & \sum_{j \in J} \sum_{i \in I} w_{i}x_{ij} + \sum_{j \in J} z_{j}y_{j} && \\
    \text{s.t. } &\sum_{j \in N_{i}}  x_{ij} \leq  d_i &\text{ for }& i \in I  \\
    & \sum_{i \in I: j \in N_{i}} x_{ij} \leq y_{j} \ &\text{ for }& j \in J \\
    & \sum_{j \in M_s}  y_{j} \leq c_s &\text{ for }& s \in S  \\
    & y_{j} \leq d_j &\text{ for }& j \in J  \\
    & x_{ij} \geq 0 \text{ }&\text{  for }& i\in I, j \in J \\
    & y_{j}  \text{ integer }&  \text{ for } &j \in J 
\end{align}
$$

In [12]:
##############################################################
## We first calculate demand for community health care workers
## Demand calculation code is written by Dare Brawley using the 
## George Washington https://www.gwhwi.org/estimator-613404.html 
## calculation
## github: darebrawley
##############################################################

In [13]:
#### replace with desired (or most recent) date in NY Times dataset:
covid_data_update_date = '2020-06-08'

#### Parameters for GW model

# base_need = contact tracers per total population 

# We consider 3 different base: Low = 30, Medium = 50, High = 70
base_need = 30

# contacts = estimated contacts per case (depends on social distancing measures)
contacts = 10

# case_interviews = number of confirmed case interviews per day  
case_interviews = 6

# cont_notifications = number of contact notifications per day  
cont_notifications = 12

# contact_followup = number of contact follow ups per day  
contact_followup = 32

# follow_freq = follow up frequency (per contact per week)  
followup_freq = 7

In [14]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [15]:
import requests
import io

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))
covid.head(10)

# We only have live url for Puerto Rico for now
#live_url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/live/us-counties.csv"

#covid[covid['state'] == "Puerto Rico"]


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
5,2020-01-25,Orange,California,6059.0,1,0
6,2020-01-25,Cook,Illinois,17031.0,1,0
7,2020-01-25,Snohomish,Washington,53061.0,1,0
8,2020-01-26,Maricopa,Arizona,4013.0,1,0
9,2020-01-26,Los Angeles,California,6037.0,1,0


In [16]:

covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)

In [17]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [18]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases']=covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
covid_last15.sort_index(inplace = True)

In [19]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()

In [20]:
## calculate follow up encounter demand
covid_last14['fe_demand'] = covid_last14['new_cases']*(14 + ((covid_last14['dt'] - data_date_dt).dt.days))/7*followup_freq*contacts

In [21]:
## group by to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases','fe_demand'].sum()

In [22]:
#svi_county.head(5)
#covid_last14_stats.head(5)
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'left', right_on = 'FIPS', left_on = 'fips' )


In [23]:
covid_last14_stats.head(5)

,fips,new_cases,fe_demand,E_TOTPOP,FIPS,STATE
0,1001,105.0,6880.0,55200,1001,ALABAMA
1,1003,61.0,5290.0,208107,1003,ALABAMA
2,1005,85.0,5370.0,25782,1005,ALABAMA
3,1007,23.0,1080.0,22527,1007,ALABAMA
4,1009,26.0,1840.0,57645,1009,ALABAMA


In [24]:
## calculations for each type of demand
covid_last14_stats['base_ct'] = (covid_last14_stats['E_TOTPOP']/100000)*base_need
covid_last14_stats['case_int_need'] = covid_last14_stats['new_cases']/case_interviews/5
covid_last14_stats['contact_notify_need'] = (covid_last14_stats['new_cases']*contacts/cont_notifications)/5
covid_last14_stats['contact_follow_need'] = covid_last14_stats['fe_demand']/(contact_followup*5*2)
covid_last14_stats['covid_need'] = covid_last14_stats['case_int_need']+covid_last14_stats['contact_notify_need']+covid_last14_stats['contact_follow_need']
covid_last14_stats['base_ct'] = covid_last14_stats['base_ct'].round()
covid_last14_stats['covid_need'] = covid_last14_stats['covid_need'].round()
covid_last14_stats['total_need'] = np.where(covid_last14_stats['covid_need'] < 1e-6, 0, covid_last14_stats['covid_need'])
#covid_last14_stats['total_need'] = max(covid_last14_stats['covid_need'],1)


In [25]:
covid_last14_stats.sort_values(by="new_cases").tail(5)

,fips,new_cases,fe_demand,E_TOTPOP,FIPS,STATE,base_ct,case_int_need,contact_notify_need,contact_follow_need,covid_need,total_need
2450,48113,3349.0,268270.0,2586552,48113,TEXAS,776.0,111.633333,558.166667,838.34375,1508.0,1508.0
2492,48201,3997.0,313220.0,4602523,48201,TEXAS,1381.0,133.233333,666.166667,978.81250,1778.0,1778.0
96,4013,5624.0,521230.0,4253913,4013,ARIZONA,1276.0,187.466667,937.333333,1628.84375,2754.0,2754.0
588,17031,9722.0,666330.0,5223719,17031,ILLINOIS,1567.0,324.066667,1620.333333,2082.28125,4027.0,4027.0
196,6037,18626.0,1342960.0,10098052,6037,CALIFORNIA,3029.0,620.866667,3104.333333,4196.75000,7922.0,7922.0


In [26]:
## saving simplified form
covid_demand = covid_last14_stats[['fips','total_need']]
covid_demand.sort_values(by="total_need").tail(100)
covid_demand.sort_values(by="fips").tail(10)
#print(len(covid_last14_stats[covid_last14_stats.total_need > 0]))
#print (len(covid_last14_stats))

,fips,total_need
2976,56027,0.0
2977,56029,0.0
2978,56031,0.0
2979,56033,1.0
2980,56035,0.0
2981,56037,3.0
2982,56039,0.0
2983,56041,7.0
2984,56043,3.0
2985,56045,0.0


In [27]:
############################################################################
############ END of demand calculation #####################################

In [28]:
#df_1 = pd.read_csv('County_based_demand.csv')

df = covid_last14_stats

In [29]:
#county_svi = pd.read_csv('Data/SVI_county_based_per_State.csv', encoding='latin-1')

In [30]:
#TextFileReader = pd.read_csv('Data/svi_2018_tracts_state_ranked.csv',chunksize=5000)

#df_svi_x = []

#for df in TextFileReader:
#    df_svi_x.append(df)
    
#df_svi = pd.concat(df_svi_x, sort=False)


In [31]:
#df_2 = pd.merge(left = df_1, right = county_svi, how = 'right', right_on = 'FIPS', left_on = 'FIPS' )

In [32]:
#df = pd.merge(left = df_2, right = df_svi, how = 'right', right_on = 'STCNTY', left_on = 'FIPS' )

#df['RPL_ThemesStates_y'] = df['RPL_ThemesStates_y'].fillna(0)

df.head(10)
df['fips'] = df['fips'].astype(int)
svi_census['STCNTY'] = svi_census['STCNTY'].astype(int)


In [33]:
#print (svi_county['FIPS'])
df_2 = pd.merge(left = df, right = svi_census, how = 'right', right_on = 'STCNTY', left_on = 'fips' )

df_2.rename(columns={'FIPS_x': 'FIPS_county', 'STATE_x':'STATE', 'FIPS_y': 'FIPS_census', 'E_TOTPOP_x' : 'E_TOTPOP_county', 'E_TOTPOP_y' : 'E_TOTPOP_census'}, inplace=True)
df_2.head(5)

,fips,new_cases,fe_demand,E_TOTPOP_county,FIPS_county,STATE,base_ct,case_int_need,contact_notify_need,contact_follow_need,covid_need,total_need,ST,STATE_y,ST_ABBR,STCNTY,COUNTY,FIPS_census,LOCATION,AREA_SQMI,E_TOTPOP_census,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,geometry,RPL_ThemesStates,RPL_Theme1States,RPL_Theme2States,RPL_Theme3States,RPL_Theme4States
0,1001.0,105.0,6880.0,55200.0,1001.0,ALABAMA,17.0,3.5,17.5,21.5,42.0,42.0,01,ALABAMA,AL,1001,Autauga,1001020100,"Census Tract 201, Autauga County, Alabama",3.790677,1923,253,779,69,765,73,218,109,35,23,31580,5144,123,46,245,52,407,100,339,87,66,44.1,382,332.0,14,49.2,0,17.0,135,56,3,13.0,6,9,0,12,11.3,5.8,3.6,2.4,31580.0,5144.0,9.4,3.6,12.7,2.9,21.2,4.4,17.6,4.5,8.6,5.7,19.9,17.1,0.8,2.7,0.0,2.2,17.3,7.2,0.4,1.7,0.8,1.2,0.0,0.6,0.4701,0.2724,0.4033,0.4649,1.6107,0.3913,0.3451,0.4188,0.7897,0.5502,2.1038,0.5597,0.3717,0.3889,0.7606,0.3735,0.0000,0.8723,0.1999,0.0696,0.0000,1.1418,0.0993,5.6169,0.2533,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,178,79,9.3,4.1,1018,0.000948,0.150082,"POLYGON ((-86.50916 32.47344, -86.50620 32.475...",0.159322,0.199153,0.341102,0.491525,0.079661
1,1001.0,105.0,6880.0,55200.0,1001.0,ALABAMA,17.0,3.5,17.5,21.5,42.0,42.0,01,ALABAMA,AL,1001,Autauga,1001020300,"Census Tract 203, Autauga County, Alabama",2.065365,3476,433,1397,92,1296,110,523,248,55,53,22527,2837,307,95,580,87,841,254,735,180,108,72.4,1170,543.2,113,127.8,18,28.6,19,24,28,33.2,99,54,0,12,15.0,6.5,3.3,3.2,22527.0,2837.0,12.8,4.0,16.7,3.3,24.2,6.7,21.2,4.2,8.3,5.5,33.7,15.1,3.3,3.8,1.3,2.0,1.4,1.8,2.2,2.6,7.6,4.1,0.0,0.3,0.6033,0.2320,0.7218,0.6086,2.1657,0.5659,0.5877,0.6384,0.9040,0.5294,2.6595,0.8729,0.5424,0.6799,1.2223,0.6429,0.3297,0.5669,0.5416,0.6300,0.0000,2.0682,0.3882,8.1157,0.6400,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,119,81,3.4,2.2,1836,0.000516,0.100175,"POLYGON ((-86.47087 32.47573, -86.46964 32.478...",0.550000,0.327966,0.760169,0.852119,0.445763
2,1001.0,105.0,6880.0,55200.0,1001.0,ALABAMA,17.0,3.5,17.5,21.5,42.0,42.0,01,ALABAMA,AL,1001,Autauga,1001020400,"Census Tract 204, Autauga County, Alabama",2.464982,3831,337,1867,106,1639,144,109,70,49,40,30527,3194,165,60,901,119,785,135,742,146,104,72.3,470,449.2,51,74.2,72,67.4,0,12,13,16.4,21,24,0,12,2.8,1.8,2.9,2.3,30527.0,3194.0,6.0,2.2,23.5,3.4,20.5,3.0,19.6,3.7,6.3,4.4,12.3,11.7,1.4,2.1,3.9,3.6,0.0,1.7,0.8,1.0,1.3,1.4,0.0,0.3,0.0609,0.1818,0.4345,0.2822,0.9594,0.1825,0.8851,0.3687,0.8616,0.3749,2.4903,0.7960,0.2456,0.5014,0.7470,0.3661,0.4731,0.0000,0.2952,0.1179,0.0000,0.8862,0.0585,5.0829,0.1805,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,108,100,2.8,2.6,1951,0.000609,0.114106,"POLYGON ((-86.45394 32.49318, -86.44742 32.493...",0.101695,0.085593,0.628814,0.480508,0.039831
3,1001.0,105.0,6880.0,55200.0,1001.0,ALABAMA,17.0,3.5,17.5,21.5,42.0,42.0,01,ALABAMA,AL,1001,Autauga,1001020500,"Census Tract 205, Autauga County, Alabama",4.404987,9883,726,4488,188,4174,285,1471,729,100,89,40235,11695,482,215,1366,192,2527,376,1429,293,288,170.0

In [34]:
# Create a dictionary for the county and population
population_county = dict(zip(svi_county.FIPS, svi_county.E_TOTPOP))

# Create a dictionary for the census tract and population
population_census = dict(zip(svi_census.FIPS, svi_census.E_TOTPOP))

# Create a dictionary for the name and FIPS of the county
FIPS_census_county_dict = dict(zip(svi_census.STCNTY, svi_census.COUNTY))

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_census.STCNTY, svi_census.STATE))

# Create a dictionary for census tract FIPS and county FIPS
census_county = dict(zip(svi_census.FIPS, svi_census.STCNTY))


# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = [k for k in SVI_county if not math.isnan(SVI_county[k])]


# Create a dictionary for the demand per county
# We use George Washington https://www.gwhwi.org/estimator-613404.html
# to calculate the demand
county_demand_14 = dict(zip(df.fips, df.total_need))

county_demand = {}

for j in location:
    if j in county_demand_14 and county_demand_14[j] > 1e-10:
        county_demand[j] = county_demand_14[j]
    else:
        county_demand[j] = 0
       

# Create demand point list from the census tract FIPS 
df_2['FIPS_census'] = df_2['FIPS_census'].astype(int)

demand_point = df_2.FIPS_census.tolist()


In [35]:
# Each census only can be covered by its county so we simple set it to coverage set to the county of that census
coverage_loc_set = census_county


# since we have the demand data in the county level, to create the demand in the census tract level, we simple use the
# ratio of the census population to the county population
demand = {}
for d in demand_point:
    
    if  population_county[census_county[d]] >= 1e-6:
        demand[d] = (county_demand[census_county[d]]*population_census[d])/population_county[census_county[d]]       
    else:
        demand[d] = 0     
    


In [36]:
#demand_point, demand = gp.multidict({demand_p:demand} for demand in demand_county for demand_p in demand_point_x)
new_df = df_2.drop_duplicates(subset=['fips'])
new_df.dropna(subset=['STATE'])
State = list(set(svi_county.STATE.tolist()))
State = [s for s in State if s != math.nan]
capacity = new_df.groupby(['STATE'])['base_ct'].sum().to_dict()


In [37]:
demand_per_State = new_df.groupby(['STATE'])['total_need'].sum().to_dict()
population_per_state =  new_df.groupby(['STATE'])['E_TOTPOP_county'].sum().to_dict()

##########################################
# Change CT needed per 100,000 population
# For we can set it to low = 30, medium = 50, 
# high = 70 CT or any number in the set {0, 10, ..., 100) CT per 100,000
capacity_multiplier = 30

######################################
# Or specify certain number of CT per state
minimum_contact_tracer_per_state = {s: 1 for s in State}
# print (minimum_contact_tracer_per_state)

capacity_x = {}
#print (State)
for s in capacity:
    capacity_x[s] = math.ceil(max((capacity_multiplier*population_per_state[s])/100000, minimum_contact_tracer_per_state[s]))
    

In [38]:
#Parameters
#location = list(dict.fromkeys(location))
pro = [(i,coverage_loc_set[i]) for i in demand_point]
#print (location)
#print (pro)
pro_c_s = [(i,county_of_states[i]) for i in location]
#print (pro_c_s)

In [39]:

cartesian_prod = gp.tuplelist(pro)
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [40]:
#MIP model formulation
m = gp.Model("Contact_Tracing_Coverage")

Using license file /Users/si272/gurobi.lic
Academic license - for non-commercial use only


In [41]:
#Add variable for each contact tracer center
y = m.addVars(location, vtype = GRB.INTEGER, name = "y")

#Add variable for each demand point (census tract)
x = m.addVars(cartesian_prod, vtype = GRB.CONTINUOUS, name = "x")



In [42]:
# Coverage constraint
m.addConstrs((gp.quicksum(x[i,j] for (i,j) in cartesian_prod.select(i,'*')) <= demand[i] for i in demand_point), name = 'census_tract_demand')
m.update()

In [43]:
#CT allocation for all demand point (census tracts) in a county cannot exceed the allocation to the county
m.addConstrs((gp.quicksum(x[i,j] for (i,j) in cartesian_prod.select('*',j)) <=   y[j] for j in location), name = 'census_tract_and_county_allocation')
m.update()

In [44]:
# Each state has a predefined capacity. The allocation of CT in all counties for each state cannot exceed the state's capacity
capacity_const = {}
for s in capacity:
    capacity_const[s] = m.addConstr((gp.quicksum(y[j] for j in location if (j,s) in cartesian_pro_county_state) <= (capacity[s])), name = 'State_capacity')
m.update()
    
#for s in capacity:
#    print (capacity_const[s])
 #   print (capacity_const[s].rhs)


In [45]:
m.addConstrs((y[j] <=   county_demand[j] for j in location), name = 'County_demand')
m.update()


In [46]:

SVI_county = {k: SVI_county[k] for k in SVI_county if not math.isnan(SVI_county[k])}

hotspot = {} #demand divided by pop if no pop then it is 0
total = 0
for j in location:
    #print (county_demand[j],j)
    
    if population_county[j] <1e-6:
        hotspot[j] = 0
    else:
        hotspot[j] = county_demand[j]/population_county[j]
        total += hotspot[j]
        #print(j, county_demand[j], demand_per_county[j], population_county[j])


######################################################################
## Normalize the weights we use for different policies
## First we need max and min for each different policy
min_demand = min(county_demand[j] for j in location)
max_demand = max(county_demand[j] for j in location)

min_SVI_hotspot = min(SVI_county[j]*hotspot[j] for j in location)
max_SVI_hotspot = max(SVI_county[j]*hotspot[j] for j in location)

min_SVI_pop = min(population_county[j]*SVI_county[j] for j in location)
max_SVI_pop = max(population_county[j]*SVI_county[j] for j in location)

min_hotspot = min(hotspot[j] for j in location)
max_hotspot = max(hotspot[j] for j in location)

high_demand_normalized = {}
SVI_hotspot_normalized = {}
SVI_pop_normalized = {}
hotspot_normalized = {}

high_demand = {}
SVI_hotspot = {}
SVI_pop = {}

#print (location)

for j in location:
    
    high_demand[j] = county_demand[j]
    
    SVI_hotspot[j] = SVI_county[j]*hotspot[j]
    
    SVI_pop[j] = population_county[j]*SVI_county[j]

    if  math.isnan (high_demand[j]) :
        print ('high demand', j)
    if math.isnan(SVI_hotspot[j]):
        print ('SVIh', j)
        
    if math.isnan(SVI_pop[j]):
        print ('SVIp', j)
    if math.isnan(hotspot[j]):
        print ('hotspot', j)
    if math.isnan(SVI_county[j]):
        print ('SVI', j)
# dict comprehension
high_demand = {k: high_demand[k] for k in high_demand if not math.isnan(k)}


for j in location:
    
    high_demand_normalized[j] = (county_demand[j] - min_demand)/(max_demand - min_demand)
    
    SVI_hotspot_normalized[j] = (SVI_county[j]*hotspot[j] - min_SVI_hotspot)/(max_SVI_hotspot - min_SVI_hotspot)
    
    SVI_pop_normalized[j] = (population_county[j]*SVI_county[j] - min_SVI_pop)/(max_SVI_pop - min_SVI_pop)
    
    hotspot_normalized[j] = (hotspot[j] - min_hotspot)/(max_hotspot - min_hotspot)
    
    #print (j, high_demand_normalized[j], SVI_hotspot_normalized[j], SVI_pop_normalized[j], hotspot_normalized[j] )

min_SVI_pop_census = min(SVI_census[i]*population_census[i] for i in demand_point)
max_SVI_pop_census = max(SVI_census[i]*population_census[i] for i in demand_point)

census_SVI_pop_normalized = {i: (SVI_census[i]*population_census[i] - min_SVI_pop_census)/(max_SVI_pop_census - min_SVI_pop_census)  for i in demand_point}


In [47]:
#print (demand)

In [48]:
# We change the weight in the objective for different policies
# (1) demand_per_county[j] prioritizes the counties with high demand first within per state to allocate contact tracers

# (2) (SVI*hotspot = SVI_county[j]*hot_stop_weight[j]*100000) this one we prioritize SVI*hot spot counties 

# (3) (SVI*pop for county = SVI_county[j]*population_county[j]) prioritize SVI and high pop areas 

# (4) hot_spot_weight[j]*100000 this one prioritize hot spot counties first

Strategies = {'high_demand': high_demand, 'SVI_hotspot': SVI_hotspot, 'SVI_pop': SVI_pop, 'hotspot': hotspot}

m.update()

percentage_census = {}
percentage_county = {}

# We consider three different level for base case capacity low = 30 per 100,000
# medium = 50 per 100,000 and high = 70 per 100,000

base_case_capacity = {'low' : 30, 'mid': 50, 'high': 70}

#print (Strategies)
for s in Strategies:
    
    #for j in location:
     #   print (s,j,Strategies[s][j])
    
    for c in base_case_capacity:
        percentage_census[s,c] = {}
        percentage_county[s,c] = {}
        
        for p in capacity:
            #print (p)
            #print (capacity_const[p].RHS)
        
            capacity_const[p].RHS = (capacity[p]/base_case_capacity['low'])*base_case_capacity[c]
    
        m.setObjective( gp.quicksum(SVI_census[i]*population_census[i]*x[i,j] for (i,j) in cartesian_prod  if not math.isnan(SVI_census[i]))
 + gp.quicksum(Strategies[s][j]*y[j] for j in location), GRB.MAXIMIZE)

        #for i in demand_point:
        #    if math.isnan(SVI_census[i]):
        #        print (i, SVI_census[i], population_census[i])
        m.update()
        #m.write("CT_coverage_model.lp")
        print ('\n************ Strategy',s,'******* Base case capacity*******', c)
        m.update()
        m.optimize()
        
        for i in demand_point:
            if demand[i] > 0:
                #print (i)
                percentage_census[s,c][i] = (100*sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))/demand[i]) 
                
            else:
                percentage_census[s,c][i] = -1
        
        
        
        for j in location:
            if county_demand[j] > 1e-6:
                percentage_county[s,c][j] = (y[j].x/county_demand[j])*100
            else:
                percentage_county[s,c][j] = -1
                
        #print (percentage[s,c])
        m.reset()



************ Strategy high_demand ******* Base case capacity******* low
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 79172 rows, 75979 columns and 155100 nonzeros
Model fingerprint: 0x14250bf7
Variable types: 72837 continuous, 3142 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-03, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-05, 1e+04]
Found heuristic solution: objective 3.288331e+08
Presolve removed 79123 rows and 67962 columns
Presolve time: 0.40s
Presolved: 49 rows, 8017 columns, 8065 nonzeros
Found heuristic solution: objective 3.998940e+08
Variable types: 7965 continuous, 52 integer (4 binary)

Root relaxation: objective 4.116667e+08, 51 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.1167e+08    0   45 3.9989e+08 4.1167e+08  2.94%    


Solution count 3: 3.47408e+08 3.45352e+08 3.43435e+08 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.474092065395e+08, best bound 3.474170628233e+08, gap 0.0023%
Discarded solution information

************ Strategy SVI_pop ******* Base case capacity******* low
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 79172 rows, 75979 columns and 155100 nonzeros
Model fingerprint: 0xe27ff4a0
Variable types: 72837 continuous, 3142 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-03, 7e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-05, 1e+04]
Found heuristic solution: objective 8.765701e+10
Presolve removed 79123 rows and 67962 columns
Presolve time: 0.23s
Presolved: 49 rows, 8017 columns, 8065 nonzeros
Found heuristic solution: objective 1.080659e+11
Variable types: 7965 continuous, 52 integer (4 binary)

Root relaxation: objective 1.095517e+11, 42 iterations, 0.00 seconds

    Nodes  


Optimal solution found (tolerance 1.00e-04)
Best objective 3.474092444381e+08, best bound 3.474171007213e+08, gap 0.0023%
Discarded solution information


In [49]:
#Write timestamp 

import time
from datetime import datetime
time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('../Community_Health_workers_Allocation/Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

NameError: name 'csv' is not defined

In [ ]:
# Write csv file for census tracts


import csv
fieldnames = []  
write_census_row = {}
fieldnames.append('Census_Tract_FIPS')
fieldnames.append('priority')
fieldnames.append('SVI_census')

SVI_values = {i:SVI_census[i] for i in demand_point}
s_count = 1
for s in Strategies:   
    for c in base_case_capacity:
        fieldnames.append('percentage_scenario_' + s + '_base_case_capacity_' + c)

        write_census_row[s_count] = (percentage_census[s,c].values())
        s_count += 1

writefile = '../Community_Health_workers_Allocation/Output/Census_tract_level_coverage_for_all_policies_and_low_mid_high_base_case_capacity.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)  
    writer.writerow(fieldnames)
    for row in zip(demand_point, census_SVI_pop_normalized.values(), SVI_values.values(), write_census_row[1], write_census_row[2], write_census_row[3],write_census_row[4], write_census_row[5], write_census_row[6], write_census_row[7]
                   , write_census_row[8],write_census_row[9],write_census_row[10],write_census_row[11],write_census_row[12]):                    
        writer.writerow(row)
            
            


In [ ]:
# Write file for county 

import csv
fieldnames = []  
write_census_row = {}
fieldnames.append('County_FIPS')
fieldnames.append('SVI_county')
fieldnames.append('priority_high_demand')
fieldnames.append('priority_SVI_hotspot')
fieldnames.append('priority_SVI_pop')
fieldnames.append('priority_hotspot')

SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    for c in base_case_capacity:
        fieldnames.append('percentage_scenario_' + s + '_base_case_capacity_' + c)

        write_census_row[s_count] = (percentage_county[s,c].values())
        s_count += 1

writefile = '../Community_Health_workers_Allocation/Output/County_level_coverage_for_all_policies_and_low_mid_high_base_case_capacity.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location, SVI_values.values(), high_demand_normalized.values(), SVI_hotspot_normalized.values(), SVI_pop_normalized.values(), hotspot_normalized.values(),  write_census_row[1], write_census_row[2], write_census_row[3],write_census_row[4], write_census_row[5], write_census_row[6], write_census_row[7]
                   , write_census_row[8],write_census_row[9],write_census_row[10],write_census_row[11],write_census_row[12]):                    
        writer.writerow(row)